In [ ]:
!ls /kaggle/input/dogs-vs-cats

# CREATE FOLDER FOR LOAD DATA

### Uncompressing data

In [ ]:
from zipfile import ZipFile as zf
train_zip = zf('/kaggle/input/dogs-vs-cats/train.zip', 'r')
train_zip.extractall()
train_zip.close()
test_zip = zf('/kaggle/input/dogs-vs-cats/test1.zip', 'r')
test_zip.extractall()
test_zip.close()

### Path to the directory where the original dataset was uncompressed

In [ ]:
import os, shutil
import random

# PATH_DATA = '/kaggle/input/dogs-vs-cats/'
BASE_DIR = './my_arange_data/'
os.makedirs(BASE_DIR, exist_ok=True)

# Make directories of train, validation, test
train_dir = os.path.join(BASE_DIR, 'train')
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(BASE_DIR, 'validation')
os.makedirs(validation_dir, exist_ok=True)

# With cats pictures
train_cats_dir = os.path.join(train_dir, 'cats')
os.makedirs(train_cats_dir, exist_ok=True)
valid_cats_dir = os.path.join(validation_dir, 'cats')
os.makedirs(valid_cats_dir, exist_ok=True)


# With dogs pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.makedirs(train_dogs_dir, exist_ok=True)
valid_dogs_dir = os.path.join(validation_dir, 'dogs')
os.makedirs(valid_dogs_dir, exist_ok=True)

# copy train to base_dir train
for filename in os.listdir('./train'):
    if filename.split('.')[0] == 'dog':
        shutil.copy(os.path.join('./train/',filename), os.path.join(train_dogs_dir, filename))
    else:
        shutil.copy(os.path.join('./train/',filename), os.path.join(train_cats_dir, filename))

        # move some of train to validation
train_dogs = os.listdir(train_dogs_dir)
train_cats = os.listdir(train_cats_dir)

random.shuffle(train_dogs)
random.shuffle(train_cats)

## train: 10000 image, valid: 2500 for each class
for i in range(2500):
    shutil.move(os.path.join(train_dogs_dir, train_dogs[i]), os.path.join(valid_dogs_dir, train_dogs[i]))
    shutil.move(os.path.join(train_cats_dir, train_cats[i]), os.path.join(valid_cats_dir, train_cats[i]))        

In [ ]:
print(len(os.listdir(train_dogs_dir)), len(os.listdir(valid_dogs_dir)))

# Data Preprocessing

### Display some pictures

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

fnames = [os.path.join('./train/', fname) for fname in os.listdir('./train')]
for i, fname in enumerate(fnames):
    if i <= 4:
        plt.figure(i)
        plt.imshow(mpimg.imread(fname))

### Load data

Using class ImageDataGenerator to load images

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

load_datagen = ImageDataGenerator(rescale=1./255)

train_generator = load_datagen.flow_from_directory(\
                                                   train_dir,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='binary')
valid_generator = load_datagen.flow_from_directory(\
                                                   validation_dir,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='binary')

# Try with simple model

Check gpu device

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(lr=1e-4),
    metrics=['acc'])

In [ ]:
model.summary()

In [ ]:

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=50, 
    validation_data=valid_generator, 
    validation_steps=50)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def visualize_result(history):
    fig, axes = plt.subplots(2, 1, figsize=(15, 12))
    epochs = np.arange(1, len(history.history['acc']) + 1, 1)
    # accuracy
    axes[0].plot(epochs, history.history['acc'], '-o', color='b', label='train_accuracy')
    axes[0].plot(epochs, history.history['val_acc'], '-o', color='r', label='val_accuracy')
    axes[0].legend()
    #loss
    axes[1].plot(epochs, history.history['loss'], '-o', color='b', label='train_loss')
    axes[1].plot(epochs, history.history['val_loss'], '-o', color='r', label='val_loss')
    axes[1].legend()

    axes[0].set_xticks(epochs)
    axes[1].set_xticks(epochs)
visualize_result(history)

### Augmentation data

You can using augmentation method to make more data from original data.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(
    rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size=128,
    target_size=(150, 150),
    class_mode='binary')
valid_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=128,
    class_mode='binary')

- Let see images generator after using augmentation for a picture

In [ ]:
from tensorflow.keras.preprocessing import image
img = image.load_img(fnames[1], target_size=(150, 150))
plt.imshow(img)

In [ ]:
x = image.img_to_array(img)
x = x.reshape((1,x.shape[0], x.shape[1], x.shape[2]))
generator_img = train_datagen.flow(x, batch_size=1)

for i, batch in enumerate(generator_img):
    plt.figure(i)
    plt.imshow(image.array_to_img(batch[0]))
    if i == 10:
        break
plt.show()

# USING PRETRAINED CONVNET

Using Xception model from keras api, you can read more in this: https://keras.io/api/applications/xception/

In [ ]:
from keras.applications import Xception

pretrained_model = Xception(
    weights='imagenet', 
    include_top=False, 
    input_shape=(150, 150, 3))

pretrained_model.summary()

notice the final feature map has shape (None, 5, 5, 2048)

## 1. Feature extraction before training

Using pretrained model to extract the features of data, then using these to predict.

In [ ]:
def extract_features(dir):
    n_samples = len(os.listdir(dir))
    features = np.zeros(shape=(n_samples, 5, 5, 2048))
    labels = np.zeros(shape=(n_samples))
    datagen = ImageDataGenerator(rescale=1./255)
    
    batch_size = 1
    # using ImageDataGenerator format
    gen_data = datagen.flow_from_directory(
        dir, 
        target_size=(150, 150), 
        batch_size=batch_size, 
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in gen_data:
        features_batch = pretrained_model.predict(inputs_batch)
        features[i*batch_size: (i+1)*batch_size] = features_batch
        labels[i*batch_size: (i+1)*batch_size] = labels_batch
        if i*batch_size > n_samples:
            break
        i += 1
    return features, labels

train_features, train_labels = extract_features(train_dir)
valid_features, valid_labels = extract_features(validation_dir)

In [ ]:
train_features = train_features.reshape(train_features.shape[0], 5*5*2048)
valid_features = valid_features.reshape(valid_features.shape[0], 5*5*2048)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=(5*5*2048)))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(
    optimizer=optimizers.RMSprop(lr=2e-5), 
    loss='binary_crossentropy', 
    metrics=['acc'])

In [ ]:
model.fit(
    train_features, 
    train_labels, 
    epochs=30, 
    batch_size=20, 
    validation_data=(valid_features, valid_labels))

## 1. training with pretrained model

Add some layers to pretrained model

In [ ]:
model = models.Sequential()
model.add(pretrained_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(
    optimizer=optimizers.RMSprop(lr=2e-5), 
    loss='binary_crossentropy', 
    metrics=['acc'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator

load_datagen = ImageDataGenerator(rescale=1./255)

train_generator = load_datagen.flow_from_directory(\
                                                   train_dir,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='binary')
valid_generator = load_datagen.flow_from_directory(\
                                                   validation_dir,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='binary')
model.fit(
    train_generator, 
    epochs=50, 
    steps_per_epoch=100, 
    validation_data=valid_generator, 
    validation_steps=50)

## 2. Fine-tuning

In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

In [ ]:
pretrained_model = Xception(
    weights='imagenet', 
    include_top=False, input_shape=(150, 150, 3))
pretrained_model.summary()

Using layers in block14 

In [ ]:
pretrained_model.trainable = True
set_trainable = False

for layer in pretrained_model.layers:
    if layer == 'block14_sepconv1':
        set_trainable = True
        
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
model = models.Sequential()
model.add(pretrained_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(
    optimizer=optimizers.RMSprop(lr=2e-5), 
    loss='binary_crossentropy', 
    metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

load_datagen = ImageDataGenerator(rescale=1./255)

train_generator = load_datagen.flow_from_directory(\
                                                   train_dir,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='binary')
valid_generator = load_datagen.flow_from_directory(\
                                                   validation_dir,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='binary')
model.fit(
    train_generator, 
    epochs=50, 
    steps_per_epoch=100, 
    validation_data=valid_generator, 
    validation_steps=50)